# Test Misc Functions
This notebook contains snippets of code used to test various functions. 

Some functions take the source data as input to perform operations. This assumes that the data processing has been done, please check paths.

Some functions generate output files, which are stored in the folder data/samples/<some name>. If this does not exist, an error will be thrown, generate this folder first.

# Reading/Writing HDF5 .h5 files
This is used to convert the output of the surface normal prediction model to suitable input format for the depth2depth module. 

In [ ]:
import h5py
import numpy as np
import os

'''This will read in an hdf5 file as reference, and save the contents back to a tmp file. After that
it reads the contensts of saved file and compares to original file.

NOTE: This is used to save surface normals in .h5 file for depth2depth module. The expected shape in (3, h, w), float32.
'''

### Reading a file
path_h5 = 'data/depth2depth_sample_files/normal_scannet_realsense_test/realsense_004_normal_est.h5'
path_h5_test = 'data/samples/surface_normal_hdf5/mytestfile.h5'
h5_key = 'result'

print('Reading hdf5 file \'{}\' ...\n'.format(path_h5))
f = h5py.File(path_h5, 'r')

f_keys = list(f.keys())
print('f_keys:', f_keys)

dset = f[h5_key]
print('dset.shape:', dset.shape)
print('dset.dtype:', dset.dtype)
print('dset.name:', dset.name)
print('---')

# Reading values from a HDF5 dataset
dset_data = dset[()]
print('datatype in dataset: ', type(dset_data))

### Writing a file
with h5py.File(path_h5_test, "w") as f:
    dset2 = f.create_dataset('/result', data=dset_data)
    

### Reading back contents of written file
with h5py.File(path_h5_test, 'r') as f:
    f_keys = list(f.keys())
    print('List keys: ', f_keys)
    dset2 = f[h5_key]
    dset2_data = dset2[:]

if np.array_equal(dset_data, dset2_data):
    print('2 arrays equal')
else:
    print('2 arrays not equal')

